In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.metrics import * # Score Criteria
import lightgbm as lgb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
X_train, y_train = pd.read_csv('/content/X_train.csv'), pd.read_csv('/content/y_train')
X_test, submission = pd.read_csv('/content/X_test.csv'), pd.read_csv('/content/sample_submission.csv')

In [ ]:
# 각 list의 값들이 같아야함.
[len(X_train), len(y_train)], [len(X_test), len(submission)], [len(X_train.columns), len(X_test.columns)]

([1952, 1952], [113, 113], [132, 132])

In [ ]:
# TODO : Check if model works better with sampling
'''
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)
'''

'\nsmote = SMOTE(random_state=42)\nX, y = smote.fit_resample(X, y)\n'

Model : GV + LGB

In [ ]:
params = {
    'iterations': [3000],
    'depth': [1, 2, 3],
    'learning_rate' : [0.001, 0.003, 0.005, 0.007, 0.009],
    'random_seed' : [42],
}

In [ ]:
GV_lgb_model = GridSearchCV(estimator=lgb.LGBMRegressor(),
                                    param_grid=params,
                                    scoring='neg_mean_squared_error', # MSE
                                    n_jobs=-1,
                                    cv=5)

In [ ]:
GV_lgb_model.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003168 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18562
[LightGBM] [Info] Number of data points in the train set: 1098, number of used features: 132
[LightGBM] [Info] Start training from score 597.002967


GridSearchCV(cv=5, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'depth': [1, 2, 3], 'iterations': [3000],
                         'learning_rate': [0.001, 0.003, 0.005, 0.007, 0.009],
                         'random_seed': [42]},
             scoring='neg_mean_squared_error')

Find Best parameters

In [ ]:
print(GV_lgb_model.score(X_train, y_train))

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: iterations
-4701937.068849385


In [ ]:
print(GV_lgb_model.best_params_)

{'depth': 1, 'iterations': 3000, 'learning_rate': 0.009, 'random_seed': 42}


In [ ]:
print(GV_lgb_model.best_score_)

-5395584.760620037


In [ ]:
y_pred = lgb_model.predict(X_test)

In [ ]:
lgb_submission = pd.DataFrame({'ID':submission['ID'], 'IC50_nM':y_pred * 0.19})
lgb_submission.to_csv('lgb_submission.csv', index=False)

- 0.2 : 0.4363586688
- 0.19 : 0.4319936327

In [ ]:
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(learning_rate=0.005, n_estimators=500, random_seed=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
lgb_model.fit(X_train, y_train )

In [ ]:
y_pred = lgb_model.predict(X_test)
n =  0.2057057057057057

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

644.7027894951808

In [ ]:
y_pred = lgb_model.predict(X_test)
len(y_pred)

113

In [ ]:
lgb_submission = pd.DataFrame({'ID':submission['ID'], 'IC50_nM':y_pred * 0.19})
lgb_submission.to_csv('lgb_submission.csv', index=False)

LGB - 02

In [ ]:
lgb_model = lgb.LGBMRegressor(num_iterations=3000, learning_rate=0.005, random_seed=42)
lgb_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002089 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18562
[LightGBM] [Info] Number of data points in the train set: 1098, number of used features: 132
[LightGBM] [Info] Start training from score 597.002967


LGBMRegressor(learning_rate=0.005, num_iterations=3000, random_seed=42)

In [ ]:
y_pred = lgb_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

16807.924977411214

In [ ]:
n_space = np.linspace(0, 0.5, 100)

min_val = 500

for i in range(len(n_space)) :
  if min_val > np.sqrt(mean_squared_error(y_test, y_pred * n_space[i])) :
    print(n_space[i])